# COGS 108 - Final Project

## Important

- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'FinalProject.ipynb'.
- Only upload the .ipynb file to TED, do not upload any associted data. Make sure that for cells in which you want graders to see output that these cells have been executed.

## Group Members: Fill in the Student IDs of each group member here

Replace the lines below to list each persons full student ID, ucsd email and full name.

- A12785571, ekfu@ucsd.edu, Edwin Fu
- A12623303, jcloo@ucsd.edu, Joel Loo
- A13589612, bfbarry@ucsd.edu, Brian Barry
- A12690911, zes004@ucsd.edu, Sunny Sun
- A13736482, roz021@ucsd.edu, Roy Zhang
- A13014470, khh007@ucsd.edu, Kevin Huang


## Introduction: Background

From 2013-2018, the United States of America has gone through a period of time when there were over 260,000 reported occurrence of gun violence. With so many reported incidents within the span of only 5 years, there needs to be a way for citizens to find where all these incidents are occurring without having to sift through heaps of data. People should be aware of where areas of high gun violence rates are occurring and how possible prevention can occur. We seek to find out whether or not gun violence rates, household income level, and population level are correlated and if so, can there be a way to lower the gun violence rates in those areas. To operationalize this, we are using gun violence rates, household income levels, and population counts as our metrics since all three can be utilized for our research question of whether there is a correlation between income and gun violence within an area of the United States.

To start with our analysis, we will start by importing some Python library packages for analyzing and graphing our data:


# Library imports

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

plotly.tools.set_credentials_file(username='jcloo', api_key='6Qa08K4DBMMNjPHL2p0T')


In [2]:
# Configure Seaborn library
sns.set()
sns.set_context('talk')

In [3]:
#read datasets
url = "https://doc-0k-88-docs.googleusercontent.com/docs/securesc/a4oc2hqjnhqph43vaufe9fs3nelfc15u/n2em47vd7bqh0bu7a279knuoq26ugn8a/1552176000000/12632076152020903193/12632076152020903193/1KXzWFA-RIXwBXuOu81-E-MYi-e7WVfTj?e=download&nonce=1r1nehh1hfp4m&user=12632076152020903193&hash=uc1qcip4tede3ao389g85jg856941gf0"
gun_data = pd.read_csv("datasets/gun-violence-data_01-2013_03-2018.csv")
income_data = pd.read_csv("datasets/kaggle_income.csv", encoding = "ISO-8859-1")
rent_data = pd.read_csv("datasets/price.csv", encoding = "ISO-8859-1")
states = pd.read_csv("datasets/states.csv")

In [4]:
#Remove null values
#gun_data.dropna(inplace = True)
#income_data.dropna(inplace = True)
gun_data_abbrev = gun_data[['state', 'city_or_county','n_killed', 'n_injured']]

## Data Description

The datasets we are using are US State Population, Gun Violence, US Household Income. Details include state, county, state population, household income, its mean, its median, and its standard deviation. We will be using these sets to find out if the frequency of gun violence has a direct correlation with household income with the effect of population density.
These datasets can be obtained on:
US State Population - https://www.kaggle.com/lucasvictor/us-state-populations-2018?fbclid=IwAR2QTVJfUDl20U0YXWy804-poFuxP1f5IDnJecWh0AMGgoJImjTS0DVNsos
Gun Violence - 
https://www.kaggle.com/jameslko/gun-violence-data
US Household Income -
https://www.kaggle.com/goldenoakresearch/us-household-income-stats-geo-locations


In [5]:
#gun_data[(gun_data["city_or_county"]=="Mckeesport") & (gun_data["state"]=="Pennsylvania")]

# # clean/merge datasets by city

In [ ]:
#TODO: match rows/merge by City
#loop through income data and aggregate a dataframe by city

city_metrics = pd.DataFrame()
count = 0

for index, income_row in income_data.iterrows():
    city = income_row['City']
    state = income_row['State_Name']
    code = income_row['State_ab']
   # print(city)
    #print(state)
    
    #get dataframe of incident rows with matching state and city
    incidents = gun_data_abbrev.loc[(gun_data['state']  == state) & (gun_data['city_or_county'] == city)]
    #print(incidents)
    #compute each city's total incidents and total deaths/ whatever metrics for violence we want to use
    num_incidents = incidents.shape[0]
    total_killed = incidents['n_killed'].sum()
    total_injured = incidents['n_injured'].sum()

    #extract income metrics for city
    mean_inc = income_row['Mean']
    median_inc = income_row['Median']
    stdev_inc = income_row['Stdev']
    
    #construct row of the values we want to describe the city
    city_row = pd.DataFrame(columns = ['state','city','code','mean_inc','median_inc','stdev_inc','num_incidents','total_killed','total_injured'])
    city_row.loc[0] = [state,city,code,mean_inc,median_inc,stdev_inc,num_incidents,total_killed,total_injured]
    #print(city_row)
    
    city_metrics = city_metrics.append(city_row)

In [ ]:
#subset rent data to take only large city (avoid duplicates)

city_metrics_rent = pd.DataFrame()
count = 0

for index, rent_row in rent_data.iterrows():
    city = rent_row['City']
    state_abbrev = rent_row['State']
    
    #convert to full state name by comparing to states list
    state = states.loc[states['Abbreviation'] == state_abbrev]['State'].values[0]
    
    print(city)
    print(state)
    
    #get dataframe of incident rows with matching state and city
    incidents = gun_data_abbrev.loc[(gun_data_abbrev['state']  == state) & (gun_data_abbrev['city_or_county'] == city)]
    
    #compute each city's total incidents and total deaths/ whatever metrics for violence we want to use
    num_incidents = incidents.shape[0]
    total_killed = incidents['n_killed'].sum()
    total_injured = incidents['n_injured'].sum()

    #extract rent metrics for city
    latest_rent = rent_row['January 2017']
    
    #construct row of the values we want to describe the city
    city_row = pd.DataFrame(columns = ['state','city', 'latest_rent','num_incidents','total_killed','total_injured'])
    city_row.loc[0] = [state,city,latest_rent,num_incidents,total_killed,total_injured]
    city_metrics_rent = city_metrics_rent.append(city_row)

In [7]:
print(city_metrics)

          state            city mean_inc median_inc stdev_inc num_incidents  \
0       Alabama       Chickasaw    38773      30506     33101             1   
0       Alabama      Louisville    37725      19528     43789             2   
0       Alabama      Columbiana    54606      31930     57348             4   
0       Alabama         Satsuma    63919      52814     47707             0   
0       Alabama  Dauphin Island    77948      67225     54270             0   
0       Alabama         Cullman    50715      42643     35886            15   
0       Alabama    East Brewton    33737      23610     28256             0   
0       Alabama         Coosada    46319      40242     38941             1   
0       Alabama             Eva    57994      39591     47235             0   
0       Alabama       Sylacauga    54807      41712     51359             9   
0       Alabama      Rainsville    47929      38947     44430             1   
0       Alabama         Gadsden    42251      28811 

In [9]:
gun_data_abbrev.loc[(gun_data_abbrev['state'] == 'Alabama') & (gun_data_abbrev['city_or_county'])]

,state,city_or_county,n_killed,n_injured
127,Alabama,Florence,0,4
272,Alabama,Montgomery,3,5
296,Alabama,Huntsville,1,0
315,Alabama,Tuscaloosa,0,3
320,Alabama,Crenshaw,1,0
383,Alabama,Gadsden,0,1
394,Alabama,Huntsville,0,0
396,Alabama,Huntsville,0,0
400,Alabama,Birmingham,1,0
465,Alabama,Huntsville,0,2


## Data Visualization

In [ ]:
#TO-DO: Fix typeerror for seaborn plot

city_metrics.plot.scatter(x = 'mean_inc', y = 'num_incidents')

#inc_incidents_scat =  sns.lmplot(x ="mean_inc", y ="num_incidents", data =city_metrics, aspect = 1.5)

In [ ]:
#heatmap of US using plotly

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

us_data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = city_metrics['code'],
    z = city_metrics['num_incidents'].astype(int),
    locationmode = 'USA-states',
    text = city_metrics['num_incidents'],
    marker = go.choropleth.Marker(line = go.choropleth.marker.Line(color = 'rgb(255,255,255)', width=2)),
    colorbar = go.choropleth.ColorBar(title = "Incidents USA")
    )]

us_layout = go.Layout(
    title = go.layout.Title(text = 'Num Incidents Test'),
    geo = go.layout.Geo(
        scope='usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255,255,255)'
        ),
    )

fig = go.Figure(data = us_data, layout = us_layout)
py.iplot(fig, filename = 'test-choropleth-map')


## Ethics and Privacy
